# TF-IDF weighting retrieval implementation for TREC-DL 2020 Passage Ranking task

In [1]:
import nltk
import string

import re

import numpy as np

import time

from scipy.sparse import csr_matrix, lil_matrix
from multiprocessing import Pool

import pickle
import gc

import os

import scipy.sparse as scipy_sparse

from datetime import datetime

In [2]:
MSMARCO_PASSAGE_COLLECTION="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/pyserini/collections/msmarco-passage/collection.tsv"

STEMMED_DOCS_FILE="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/ia368v_dd_class_02/indexes/tempfile.pkl"

STEMMED_DOCS_FILE_FORMAT="{}_{}.pkl"
STEMMED_DOCS_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/ia368v_dd_class_02/indexes"

REVERSED_INDEX_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/ia368v_dd_class_02/indexes"
REVERSED_INDEX_FILE_FORMAT="reversed_index_TREC-DL_2020_{}.pkl"

REVERSED_INDEX_FILE="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/ia368v_dd_class_02/indexes/reversed_index_TREC-DL_2020_complete.pkl"

TOKENS_COUNT_FILE="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/ia368v_dd_class_02/indexes/tokens_count_TREC-DL_2020.pkl"

TFIDF_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/ia368v_dd_class_02/indexes"
TFIDF_FILE_FORMAT="tfidf_TREC-DL_2020_{}.pkl"

TFIDF_FILE="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/ia368v_dd_class_02/indexes/tfidf_TREC-DL_2020_complete.pkl"

TREC_DL_2020_QUERIES="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/pyserini/collections/trec-dl_2020-passage/msmarco-test2020-queries.tsv"
TREC_DL_2020_QRELS="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/pyserini/collections/trec-dl_2020-passage/2020qrels-pass.txt"

TREC_DL_2020_RUN_FORMAT="TREC_DL_2020_tfidf_run_{}.tsv"
TREC_DL_2020_RUN_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/ia368v_dd_class_02/runs"

TREC_EVAL_FULLPATH="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval"

NUMBER_OF_DOCS=8841822

PARTIAL_STEMMED_DOCS_COUNT=1000000

PARTIAL_TFIDF_TOKENS_COUNT=500000

## Text preprocessing functions

In [23]:
stop_words = set(nltk.corpus.stopwords.words('english'))
punctuation = set(string.punctuation)
stemmer = nltk.stem.PorterStemmer()

In [24]:
def test_preprocess_text(which_text):

    all_tokens = nltk.word_tokenize(which_text.lower())
    cleaned_tokens = [token for token in all_tokens if token not in stop_words and token not in punctuation]
    stemmed_tokens = [stemmer.stem(token) for token in cleaned_tokens]

    return stemmed_tokens

## If has already computed the tf-idf values, load then...

In [ ]:
with open(TFIDF_FILE, 'rb') as inputFile:
    all_tfidfs = pickle.load(inputFile)

## ...otherwise, compute the td-idf of each document in the corpus

Since we already have the reversed index built on the same corpus with the terms count in each document (this was done in the [boolean retrieval notebook](https://github.com/eduseiti/ia368v_dd_class_02/blob/main/boolean_retrieval_TREC-DL_2020.ipynb)), we can leverage that to compute the tf-idf or each document.

However, we don't have in hand the total number of terms/tokens for each document. Hence, need to compute it first.

### If has already computed the total number of tokens in each document, read that information....

In [3]:
with open(TOKENS_COUNT_FILE, 'rb') as inputFile:
    tokens_per_document = pickle.load(inputFile)

### ...otherwise, start computing it

Once again, we will build on the results of the [boolean retrieval notebook](https://github.com/eduseiti/ia368v_dd_class_02/blob/main/boolean_retrieval_TREC-DL_2020.ipynb).

In [ ]:
def compute_total_number_of_terms_per_document(file_index):
    
    total_number_of_terms = {}
    
    with open(os.path.join(STEMMED_DOCS_FOLDER, STEMMED_DOCS_FILE_FORMAT.format("temp", file_index)), "rb") as inputFile:
        temp_file = pickle.load(inputFile)

        read_documents_index = temp_file['doc_ids']
        stemmed_documents = temp_file['stemmed_docs']                

    print("Document range: {} until {}".format(read_documents_index[0], read_documents_index[-1]))
    
    for doc_index in range(len(stemmed_documents)):
        total_number_of_terms[int(read_documents_index[doc_index])] = len(stemmed_documents[doc_index])
    
    return total_number_of_terms

In [ ]:
with Pool(processes=3) as pool:
    terms_count = pool.map(compute_total_number_of_terms_per_document, range(0, NUMBER_OF_DOCS // PARTIAL_STEMMED_DOCS_COUNT + 1))

### Merge into a single dictionary

In [ ]:
tokens_per_document = terms_count[0]

for partial_dic in terms_count[1:]:
    for document, count in partial_dic.items():
        tokens_per_document[document] = count

In [ ]:
len(tokens_per_document)

In [ ]:
del terms_count

In [ ]:
with open(TOKENS_COUNT_FILE, 'wb') as outputFile:
    pickle.dump(tokens_per_document, outputFile, pickle.HIGHEST_PROTOCOL)

### Load the reversed index to get the term counts per document

In [4]:
with open(REVERSED_INDEX_FILE, 'rb') as inputFile:
    reversed_indexes = pickle.load(inputFile)

### Now, process each term to compute the related documents tf-idf value

Once again, due to RAM limitations, the processing will first create partial tfidf file which will be concatenated after.

In [5]:
def compute_tfidf(token, verbose=False):
    
    token_tdidfs = lil_matrix((1, NUMBER_OF_DOCS), dtype=np.float32)
    
    related_docs = reversed_indexes[token].nonzero()
    
    if verbose:
        print(related_docs)
    
    if type(reversed_indexes[token]) is scipy_sparse._csr.csr_matrix:
        related_docs_counts = np.array(reversed_indexes[token][related_docs])[0]
    else:
        related_docs_counts = np.array(reversed_indexes[token][related_docs].todense())[0]

    if verbose:
        print(related_docs_counts)

    term_idf = np.log10(NUMBER_OF_DOCS / len(related_docs[1]))

    if verbose:
        print(term_idf)
    
    for i, doc in enumerate(related_docs[1]):
        
        if verbose:
            print("i={}, doc={}".format(i, doc))
        
        doc_tf = related_docs_counts[i] / tokens_per_document[doc]
        
        if verbose:
            print(doc_tf)
        
        token_tdidfs[0, doc] = doc_tf * term_idf
        
    return token_tdidfs.tocsr()

In [6]:
def compute_tfidfs_for_tokens(tokens_list, part_index):
    
    print("len(tokens_list)={}, part_index={}".format(len(tokens_list), part_index))
    
    computed_tfidfs = {}
    
    for token in tokens_list:
        computed_tfidfs[token] = compute_tfidf(token)
        
    with open(os.path.join(TFIDF_FOLDER, TFIDF_FILE_FORMAT.format(part_index)), 'wb') as outputFile:
        pickle.dump(computed_tfidfs, outputFile, pickle.HIGHEST_PROTOCOL)
        
    return True

In [7]:
all_tokens = list(reversed_indexes.keys())

In [8]:
len(all_tokens)

3750155

### Break the existing tokens set into chunks....

In [9]:
token_groups = []

for i in range(len(reversed_indexes) // PARTIAL_TFIDF_TOKENS_COUNT + 1):
    token_groups.append(all_tokens[i * PARTIAL_TFIDF_TOKENS_COUNT:(i * PARTIAL_TFIDF_TOKENS_COUNT + PARTIAL_TFIDF_TOKENS_COUNT)])

### ...and process them in parallel

In [10]:
start_time=time.time()

with Pool(processes=2) as pool:
    results = pool.starmap(compute_tfidfs_for_tokens, zip(token_groups, range(len(token_groups))))
    
print(time.time() - start_time)

len(tokens_list)=500000, part_index=0
len(tokens_list)=500000, part_index=1
len(tokens_list)=500000, part_index=2
len(tokens_list)=500000, part_index=3
len(tokens_list)=500000, part_index=4
len(tokens_list)=500000, part_index=5
len(tokens_list)=500000, part_index=6
len(tokens_list)=250155, part_index=7
1676.556004524231


In [11]:
del reversed_indexes

In [13]:
del all_tokens

In [15]:
gc.collect()

327

### Now merge the partial tfidfs

In [16]:
with open(os.path.join(TFIDF_FOLDER, TFIDF_FILE_FORMAT.format(0)), 'rb') as inputFile:
    all_tfidfs = pickle.load(inputFile)

In [17]:
for i in range(1, 8):
    with open(os.path.join(TFIDF_FOLDER, TFIDF_FILE_FORMAT.format(i)), 'rb') as inputFile:
        partial_tfidfs = pickle.load(inputFile)
        
    for token, token_docs in partial_tfidfs.items():
        all_tfidfs[token] = token_docs

In [18]:
len(all_tfidfs)

3750155

In [19]:
with open(TFIDF_FILE, 'wb') as outputFile:
    pickle.dump(all_tfidfs, outputFile, pickle.HIGHEST_PROTOCOL)

## Run the queries

In [20]:
query_index = []
query_text = []

### Read all the queries in memory

In [21]:
with open(TREC_DL_2020_QUERIES, 'r', encoding="utf-8") as inputFile:
    for line in inputFile:
        query_data = line.split('\t')
        
        query_index.append(query_data[0])
        query_text.append(query_data[1])

len(query_text)

200

### Preprocess the queries the same way as the documents

In [25]:
with Pool(processes=8) as pool:
    stemmed_queries = pool.map(test_preprocess_text, query_text)

len(stemmed_queries)

200

### Find the query matches

Here each query/document score will be computed summing all the tf-idf values of the query/document common terms. This is the [tf-idf weighting scheme](https://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html#:~:text=The%20tf%2Didf%20weighting%20scheme,power%20to%20those%20documents).

In [26]:
def find_related_documents(stemmed_query, only_all_terms=False):
    
    related_documents = {}
    
    for token in stemmed_query:
        if token in all_tfidfs:
            
#             print(token)
            
            related_docs = all_tfidfs[token].nonzero()
            
            if type(all_tfidfs[token]) is scipy_sparse._csr.csr_matrix:
                related_docs_scores = np.array(all_tfidfs[token][related_docs])[0]
            else:
                related_docs_scores = np.array(all_tfidfs[token][related_docs].todense())[0]
            
            for i, doc in enumerate(related_docs[1]):
                if doc not in related_documents:
                    related_documents[doc] = 0

                related_documents[doc] += related_docs_scores[i]
            
    return related_documents

In [27]:
process_start_time = time.time()

queries_matches = []

for i in range(200):
    start_time = time.time()

    print("{} : {}".format(i, stemmed_queries[i]))
    
    queries_matches.append(find_related_documents(stemmed_queries[i]))

    print("{}\n".format(time.time() - start_time))
    
print("Total time to process the queries: {}\n".format(time.time() - process_start_time))    

0 : ['aziz', 'hashim']
0.007018566131591797

1 : ['rep', 'scalis']
0.009361982345581055

2 : ['kill', 'nichola', 'ii', 'russia']
0.18786287307739258

3 : ['own', 'barnhart', 'crane']
0.06454944610595703

4 : ['said', 'one', 'make', 'feel', 'inferior']
3.1322460174560547

5 : ['sing', 'monk', 'theme', 'song']
0.14347076416015625

6 : ['highest', 'career', 'passer', 'rate', 'nfl']
0.6372008323669434

7 : ['hunter', 'pattern', 'shotgun']
0.08426856994628906

8 : ['place', 'scalp', 'feel', 'sore']
0.7669088840484619

9 : ['pete', 'rose', 'ban', 'hall', 'fame']
0.10417008399963379

10 : ['thoma', 'cooley']
0.0345611572265625

11 : ['definit', 'endors']
0.374706506729126

12 : ['hormon', 'increas', 'calcium', 'level', 'blood']
1.195464849472046

13 : ['defin', 'geon']
0.17954468727111816

14 : ['amazon', 'rainforest', 'locat']
0.5901432037353516

15 : ['four', 'forc', 'act', 'airplan', 'equilibrium']
0.6663906574249268

16 : ['defin', 'pareto', 'chart', 'statist']
0.33734774589538574

17 : [

0.6068778038024902

135 : ['averag', 'wed', 'dress', 'alter', 'cost']
1.0542082786560059

136 : ['project', 'definit']
0.524193286895752

137 : ['barclay', 'fca', 'number']
0.6811261177062988

138 : ['benefit', 'polici', 'layoff']
0.3353128433227539

139 : ['hour', 'clinic']
0.5566520690917969

140 : ['symptom', 'shingl']
0.3327372074127197

141 : ['biggest', 'loser', 'challeng']
0.1307082176208496

142 : ['villag', 'burnham']
0.04504799842834473

143 : ['vitamin', 'e', 'anti', 'scar']
0.1783740520477295

144 : ['weather', 'antigua', 'novemb']
0.2709076404571533

145 : ['weather', 'novi', 'sad']
0.17457079887390137

146 : ['best', 'food', 'lower', 'cholesterol']
1.220409631729126

147 : ['carvedilol', 'use']
2.4587414264678955

148 : ['caus', 'bruis', 'appear']
1.0553312301635742

149 : ['caus', 'muscl', 'tear']
1.0187022686004639

150 : ['counti', 'dexter', 'michigan']
0.32584428787231445

151 : ['counti', 'new', 'york', 'new', 'york']
1.5975122451782227

152 : ['counti', 'rio', 'hond

### Now save the results in the TREC format

In [28]:
TREC_RESULT_LINE_FORMAT="{}\tQ0\t{}\t{}\t{}\tboolean\n"
MAX_RESULTS_TO_SAVE=1000

In [29]:
def generate_trec_format(queries_matches, query_ids, output_filename, verbose=False):
    
    with open(output_filename, 'w') as outputFile:
        for i, query_result in enumerate(queries_matches):
            
            if verbose:
                print("Saving query {}\n{}:".format(i, query_text[i]))
            
            relevant_docs = np.array(list(query_result.keys()))
            relevant_docs_scores = np.array(list(query_result.values()))
            
            relevant_docs_order = np.argsort(relevant_docs_scores)[::-1]
            
            if verbose:
                print("relevant_docs.shape={}".format(relevant_docs.shape))
            
            relevant_docs_final_result = relevant_docs[relevant_docs_order]
            relevant_docs_final_score = relevant_docs_scores[relevant_docs_order]
            
            if verbose:
                print("relevant_docs_final_result: {}\n\n".format(relevant_docs_final_result))
            
            for j, each_match in enumerate(relevant_docs_final_result[:MAX_RESULTS_TO_SAVE]):
                outputFile.write(TREC_RESULT_LINE_FORMAT.format(query_ids[i], each_match, j, relevant_docs_final_score[j]))

In [30]:
run_filename = os.path.join(TREC_DL_2020_RUN_FOLDER, TREC_DL_2020_RUN_FORMAT.format(datetime.now().strftime("%Y%m%d_%H%M%S")))

In [31]:
generate_trec_format(queries_matches, query_index, run_filename)

### Now, apply the TREC metrics

In [32]:
!{TREC_EVAL_FULLPATH} -c -mrecall.1000 -mmap -mndcg_cut.10 -mrecip_rank \
    {TREC_DL_2020_QRELS} {run_filename}

map                   	all	0.0983
recip_rank            	all	0.3632
recall_1000           	all	0.5019
ndcg_cut_10           	all	0.1740
